# $\beta$ -Pic b planet

I'll try to obtain the same posteriors as found in the paper of [Sun et al](https://arxiv.org/pdf/2201.08506.pdf)

## Import the necessary packages

In [1]:
from prior import prior_distributions
from simulator import OrbitCreator
from training import train_sbi

import torch

import orbitize
from orbitize import read_input

from lampe.data import H5Dataset
from lampe.inference import * # Import all methods and their losses

## Define the priors
The priors are defined as such 
$$
\begin{align*}
a &\sim \log \mathcal{U}(10,10^{4})\\
e &\sim \mathcal{U}(10^{8}, 0.99)\\
i &\sim \mathcal{U}(0,180)\\
\omega &\sim \mathcal{U}(0,360)\\
\Omega &\sim \mathcal{U}(0,360)\\
\tau &\sim \mathcal{U}(0,1)\\
\pi &\sim \mathcal{N}(56.95, 0.26) \\
M_T &\sim \mathcal{N}(1.22,0.08)\\
\end{align*}
$$


In [2]:
prior = prior_distributions(log_uniform_lower = torch.tensor(10.0), 
                            log_uniform_upper = torch.tensor(10**4),
                            uniform_lower = torch.tensor([10e-8, 0.0, 0.0, 0.0, 0.0]), 
                            uniform_upper = torch.tensor([0.99, 180.0, 360.0, 360.0, 1.0]),
                            gaussian_mean = torch.tensor([56.95, 1.22]), 
                            gaussian_std = torch.tensor([0.26, 0.08]))

## Instantiate the simulator

And test if it works well

In [3]:
simulator = OrbitCreator(data_set)

thetas = prior.sample((1,))
x = simulator(thetas)

label_print = ['a', 'e', 'i', 'ω', 'Ω', 'τ', 'π', 'Mt']

for label, theta_value in zip(label_print, thetas[0].tolist()):
    print(f"{label}: {theta_value:.3f}")

print("\n x:")
for i in range(0, len(x[0]), 2):
    x_coord = round(x[0][i].item(), 3)
    y_coord = round(x[0][i+1].item(), 3)
    print(f"({x_coord}, {y_coord})", end="\n")

NameError: name 'data_set' is not defined

## Load the generated dataset

In [ ]:
trainset = H5Dataset('Datasets/beta-pic-train.h5',
                     batch_size=2048, shuffle=True)
validset = H5Dataset('Datasets/beta-pic-val.h5',
                     batch_size=2048)
testset = H5Dataset('Datasets/beta-pic-test.h5')

## Train the SBI model

In [ ]:
len_obs = len(data_set) * 2 # 2 coordinates per observation

estimator = NRE(8, len_obs, hidden_features=[256]*5).cuda()

train_sbi('NRE',
          estimator, 
          NRELoss, 
          trainset, 
          validset, 
          epochs = 128)

  0%|          | 0/128 [00:00<?, ?epoch/s]

  0%|          | 0/128 [00:00<?, ?epoch/s]

Theta shape: torch.Size([2048, 8])
X shape: torch.Size([2048, 70])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x78 and 76x256)